# Station Data
This notebook will read in the csv data from multiple files, 
one file per month of 2018 and write a relational files for 

stations, (Data/2018/stations.csv) 

Trip information is divided into:

trip starts "Data/2018/startdata.csv" and 
trip ends "Data/2018/enddata.csv",

The division of start and end is done so that the info can be aggregated for number of trips for each station

Data for each is cleaned inseparate functions 
Each month is read in by a separate code box so you are able to read in only the months desired to be appended.
Stations have been added to the system over the time period so the stations for each month are also appended with duplicates 
removed

# Start Station

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime

# clean_station function


In [ ]:
clean_station_df=pd.DataFrame(columns=["stationid","stationname","stationlatitude","stationlongitude"])

In [ ]:
def clean_station(bike_df, full_station_df):
###########################################
#Selects unique stations, 
#changes column names to remove spaces
#appends to previous stations
# removes duplicates
# returns ["stationid","stationname","stationlatitude","stationlongitude"]
######################################################

    #select needed columns
    station_df=bike_df.loc[:,["start station id","start station name","start station latitude","start station longitude"]]
   
    station_df=station_df.drop_duplicates()
    station_df=station_df.dropna()
    station_df=station_df.rename(columns = {'start station id':'stationid','start station name':'stationname','start station latitude':'stationlatitude','start station longitude':'stationlongitude'})
    station_df['stationid']=station_df['stationid'].astype(int)
    
    #append
    result = pd.concat([full_station_df,station_df],sort=True)
    result =result.drop_duplicates()
    return result

# clean_start_data function


In [2]:
def clean_start_data(year,bike_df,full_data_df):
#####################################################
#  bike_df new data
# full_data_df  previously appended data
#changes column names to remove spaces
# parses starttime for startday and start month
# ensures station id is integer
# aggregates the number of trips by station month and day to starttripcount
#
# returns ['startstationid', 'startmonth','startday','starttripcount']
#################################################
    #select needed columns
    data_df=bike_df.loc[:,["starttime","start station id","bikeid"]]
    data_df=data_df.dropna()

     #parse date and capture date info for groupby
    datetime_list= [datetime.strptime( entry,"%Y-%m-%d %H:%M:%S.%f") for entry in data_df["starttime"]]

    data_df["startday"]= [datetime.strftime(entry,"%d") for entry in datetime_list]
    data_df["startmonth"]= [datetime.strftime(entry,"%m") for entry in datetime_list]
#    data_df['Start Hour']=[datetime.strftime(entry,"%H") for entry in datetime_list]
 
    data_df=data_df.rename(columns={'start station id':'startstationid'})
    data_df['startstationid']=data_df['startstationid'].astype(int)
 
    #group by station and start date - count bikeids 
    group_data_df=data_df.groupby(['startstationid', 'startmonth','startday'], as_index=False).agg( {'bikeid':'count'}) 
    
    #rename aggregate columns 
    group_data_df=group_data_df.rename(columns={'bikeid':'starttripcount'})

    #append
    return pd.concat([full_data_df,group_data_df],sort=True)

# clean_end_data function

In [3]:
def clean_end_data(bike_df,full_data_df):
#####################################################
#  bike_df new data
# full_data_df  previously appended data
#changes column names to remove spaces
# parses stoptime for endday and endmonth
# ensures station id is integer
# aggregates the number of trips by station, month and day to endtripcount
#
# returns ['endstationid', 'endmonth','endday','endtripcount']
#################################################
    #select needed columns
    data_df=bike_df.loc[:,["stoptime","end station id","bikeid"]]
    data_df=data_df.dropna()
    
     #parse date and capture date info for groupby
    datetime_list= [datetime.strptime( entry,"%Y-%m-%d %H:%M:%S.%f") for entry in data_df["stoptime"]]

    data_df["endday"]= [datetime.strftime(entry,"%d") for entry in datetime_list]
    data_df["endmonth"]= [datetime.strftime(entry,"%m") for entry in datetime_list]
#     data_df['End Hour']=[datetime.strftime(entry,"%H") for entry in datetime_list]
     
    data_df=data_df.rename(columns={'end station id':'endstationid'})
    data_df['endstationid']=data_df['endstationid'].astype(int)
   
    #group by station and start date - count bikeids and average tripdurations
    group_data_df=data_df.groupby(['endstationid', 'endmonth','endday'], as_index=False).agg( {'bikeid':'count'}) 
    #rename aggregate columns
    group_data_df=group_data_df.rename(columns={'bikeid':'endtripcount'})

    #append
    return pd.concat([full_data_df,group_data_df],sort=True)

# start processing files

In [4]:

start_data_df=pd.DataFrame(columns=['startstationid', 'startmonth','startday','starttripcount'])
end_data_df=pd.DataFrame(columns=['endstationid', 'endmonth','endday','endtripcount'])

In [5]:
#Read in January data
bike_df=pd.read_csv("Data/Sources/201801-citibike-tripdata.csv")
clean_station_df=clean_station(bike_df,clean_station_df)
start_data_df=clean_start_data(2018,bike_df,start_data_df)
end_data_df=clean_end_data(bike_df,end_data_df)

print("start",start_data_df.shape)
print("end",end_data_df.shape)
start_data_df.tail()

start (22384, 4)
end (22527, 4)


,startday,startmonth,startstationid,starttripcount
22379,27,01,3664,14
22380,28,01,3664,18
22381,29,01,3664,56
22382,30,01,3664,40
22383,31,01,3664,47


In [6]:
end_data_df.tail()

,endday,endmonth,endstationid,endtripcount
22522,27,01,3664,15
22523,28,01,3664,18
22524,29,01,3664,56
22525,30,01,3664,39
22526,31,01,3664,47


In [ ]:
#append Feb data
bike_df=pd.read_csv("Data/Sources/201802-citibike-tripdata.csv")
#clean_station_df=clean_station(bike_df,clean_station_df)
start_data_df=clean_start_data(2018,bike_df,start_data_df)
end_data_df=clean_end_data(bike_df,end_data_df)

print("start",start_data_df.shape)
print("end",end_data_df.shape)
start_data_df.tail()

In [ ]:
end_data_df.tail()

In [ ]:
#append March data
bike_df=pd.read_csv("Data/Sources/201803-citibike-tripdata.csv")
#clean_station_df=clean_station(bike_df,clean_station_df)
start_data_df=clean_start_data(2018,bike_df,start_data_df)
end_data_df=clean_end_data(bike_df,end_data_df)

start_data_df.tail()

In [ ]:
end_data_df.tail()

In [7]:
#append April data
bike_df=pd.read_csv("Data/Sources/201804-citibike-tripdata.csv")
#clean_station_df=clean_station(bike_df,clean_station_df)
start_data_df=clean_start_data(2018,bike_df,start_data_df)
end_data_df=clean_end_data(bike_df,end_data_df)

print("start",start_data_df.shape)
print("end",end_data_df.shape)
start_data_df.tail()

start (44708, 4)
end (44983, 4)


,startday,startmonth,startstationid,starttripcount
22319,26,04,3680,89
22320,27,04,3680,43
22321,28,04,3680,17
22322,29,04,3680,18
22323,30,04,3680,65


In [8]:
end_data_df.tail()

,endday,endmonth,endstationid,endtripcount
22451,27,04,3680,39
22452,28,04,3680,17
22453,29,04,3680,20
22454,30,04,3680,63
22455,27,04,3681,1


In [ ]:
#append May data
bike_df=pd.read_csv("Data/Sources/201805-citibike-tripdata.csv")
#clean_station_df=clean_station(bike_df,clean_station_df)
start_data_df=clean_start_data(2018,bike_df,start_data_df)
end_data_df=clean_end_data(bike_df,end_data_df)

print("start",start_data_df.shape)
print("end",end_data_df.shape)
start_data_df.tail()

In [ ]:
end_data_df.tail()

In [ ]:
#append June data
bike_df=pd.read_csv("Data/Sources/201806-citibike-tripdata.csv")
#clean_station_df=clean_station(bike_df,clean_station_df)
start_data_df=clean_start_data(2018,bike_df,start_data_df)
end_data_df=clean_end_data(bike_df,end_data_df)

print("start",start_data_df.shape)
print("end",end_data_df.shape)
start_data_df.tail()

In [ ]:
end_data_df.tail()

In [9]:
#append July data
bike_df=pd.read_csv("Data/Sources/201807-citibike-tripdata.csv")
#clean_station_df=clean_station(bike_df,clean_station_df)
start_data_df=clean_start_data(2018,bike_df,start_data_df)
end_data_df=clean_end_data(bike_df,end_data_df)

print("start",start_data_df.shape)
print("end",end_data_df.shape)
start_data_df.tail()

start (67904, 4)
end (68475, 4)


,startday,startmonth,startstationid,starttripcount
23191,27,07,3697,10
23192,28,07,3697,72
23193,29,07,3697,74
23194,30,07,3697,83
23195,31,07,3697,120


In [10]:
end_data_df.tail()

,endday,endmonth,endstationid,endtripcount
23487,27,07,3697,20
23488,28,07,3697,72
23489,29,07,3697,79
23490,30,07,3697,103
23491,31,07,3697,126


In [ ]:
#append Aug data
bike_df=pd.read_csv("Data/Sources/201808-citibike-tripdata.csv")
#clean_station_df=clean_station(bike_df,clean_station_df)
start_data_df=clean_start_data(2018,bike_df,start_data_df)
end_data_df=clean_end_data(bike_df,end_data_df)

print(clean_station_df.shape)
start_data_df.tail()

In [ ]:
#append Sept data
bike_df=pd.read_csv("Data/Sources/201809-citibike-tripdata.csv")
clean_station_df=clean_station(bike_df,clean_station_df)
start_data_df=clean_start_data(2018,bike_df,start_data_df)
end_data_df=clean_end_data(bike_df,end_data_df)

print(clean_station_df.shape)
start_data_df.tail()

In [ ]:
#append Oct data
bike_df=pd.read_csv("Data/Sources/JC-201810-citibike-tripdata.csv")
clean_station_df=clean_station(bike_df,clean_station_df)
start_data_df=clean_start_data(2018,bike_df,start_data_df)
end_data_df=clean_end_data(bike_df,end_data_df)

print(clean_station_df.shape)
start_data_df.tail()

In [ ]:
#append Nov data
bike_df=pd.read_csv("Data/Sources/201811-citibike-tripdata.csv")
#clean_station_df=clean_station(bike_df,clean_station_df)
start_data_df=clean_start_data(2018,bike_df,start_data_df)
end_data_df=clean_end_data(bike_df,end_data_df)

print(clean_station_df.shape)
start_data_df.tail()

In [ ]:
#append Dec data
bike_df=pd.read_csv("Data/Sources/201812-citibike-tripdata.csv")
clean_station_df=clean_station(bike_df,clean_station_df)
start_data_df=clean_start_data(2018,bike_df,start_data_df)
end_data_df=clean_end_data(bike_df,end_data_df)

print(clean_station_df.shape)
start_data_df.tail()

# final data type fix

In [11]:
start_data_df['startstationid']=start_data_df['startstationid'].astype(int)
end_data_df['endstationid']=end_data_df['endstationid'].astype(int)
#clean_station_df['station_id']=clean_station_df['station_id'].astype(int)

In [12]:
start_data_df['startmonth']=start_data_df['startmonth'].astype(int)
start_data_df['startday']=start_data_df['startday'].astype(int)

end_data_df['endmonth']=end_data_df['endmonth'].astype(int)
end_data_df['endday']=end_data_df['endday'].astype(int)

In [13]:
start_data_df['startstationid'].dtype

dtype('int32')

In [14]:
end_data_df.tail()

,endday,endmonth,endstationid,endtripcount
23487,27,7,3697,20
23488,28,7,3697,72
23489,29,7,3697,79
23490,30,7,3697,103
23491,31,7,3697,126


# Write Merged data to csv files

In [ ]:
# for start and end stations independently
#clean_station_df.to_csv("Data/2018/stations.csv", index=False,header=True)
start_data_df.to_csv("Data/2018/startdata.csv", index=False,header=True,encoding='utf-8')
end_data_df.to_csv("Data/2018/enddata.csv", index=False,header=True,encoding='utf-8')

In [15]:
# for map with join only January, April and July are used for map join data
#clean_station_df.to_csv("Data/2018/stations.csv", index=False,header=True)
start_data_df.to_csv("Data/2018/startjoindata.csv", index=False,header=True,encoding='utf-8')
end_data_df.to_csv("Data/2018/endjoindata.csv", index=False,header=True,encoding='utf-8')


# Remove stray month data 

Stray month data appears when months are chosen selectively
It can be cleaned here as needed

In [20]:
start_df=pd.read_csv("Data/2018/startdata.csv")
end_df=pd.read_csv("Data/2018/enddata.csv")

In [ ]:
start_df=pd.read_csv("Data/2018/startjoindata.csv")
end_df=pd.read_csv("Data/2018/endjoindata.csv")

In [25]:
# delete selected months as dirty data
print(start_df.shape)
start_df=start_df.loc[start_df['startmonth'] < 8,:]
start_df.shape

(156908, 4)


(156908, 4)

In [24]:
# delete selected months as dirty data
print(end_df.shape)
end_df=end_df.loc[end_df['endmonth'] < 8,:]
end_df.shape

(158543, 4)


(158296, 4)

In [26]:
# for start and end stations independently
#clean_station_df.to_csv("Data/2018/stations.csv", index=False,header=True)
start_df.to_csv("Data/2018/startdata.csv", index=False,header=True,encoding='utf-8')
end_df.to_csv("Data/2018/enddata.csv", index=False,header=True,encoding='utf-8')

In [ ]:
# for start and end stations independently
#clean_station_df.to_csv("Data/2018/stations.csv", index=False,header=True)
start_df.to_csv("Data/2018/startjoindata.csv", index=False,header=True,encoding='utf-8')
end_df.to_csv("Data/2018/endjoindata.csv", index=False,header=True,encoding='utf-8')